In [ ]:
# преобразование входв и выходов
from sklearn.decomposition import PCA

pca = PCA()
X_data = pca.fit_transform(X=X_data[[feat for feat in X_data.columns[0:5]]])
dict1 = {'AT': list(X_data[:, 0]), 'V': list(X_data[:, 1]),
         'AP': list(X_data[:, 2]), 'RH': list(X_data[:, 3]), 
         'PE': list(X_data[:, 4])}
X_data = pd.DataFrame(dict1)
print(X_data.info())

a=False
if a:
    general_visual(X_data, RES_DIR+'after_pca/'+'after_')

# очистка данных
# обнаружение, устранение дубликатов
X_data=X_data.drop_duplicates(keep='first')
print(X_data.info())
# -ii-ii- выбросов
for feat in X_data.columns:
    q1 = X_data[feat].quantile(0.99)
    q2 = X_data[feat].quantile(0.01)
    X_data=X_data[X_data[feat] < q1]
    X_data=X_data[X_data[feat] > q2]
print(X_data.info())
# пропущенные значения
X_data=X_data.dropna()
print(X_data.info())

a=False
if a:
    general_visual(X_data, RES_DIR+'after_pca/'+'after_clean_')

In [ ]:
from sklearn.preprocessing import normalize

X_data = normalize(X_data, norm='l2')
dict1 = {'AT': list(X_data[:, 0]), 'V': list(X_data[:, 1]),
         'AP': list(X_data[:, 2]), 'RH': list(X_data[:, 3]), 
         'PE': list(X_data[:, 4])}
X_data = pd.DataFrame(dict1)

a=True
if a:
    general_visual(X_data, RES_DIR+'after_norm/')

In [ ]:
from sklearn.preprocessing import Normalizer  
from scipy.stats import shapiro

for feat in X_data.columns:
    print(feat+":\n", shapiro(X_data[feat].values.reshape(-1, 1)))
    print(shapiro(Normalizer().fit_transform(X_data[feat].values.reshape(-1, 1))))
# ВЫВОД: 

X_data = Normalizer().fit_transform(X_data)
dict1 = {'AT': list(X_data[:, 0]), 'V': list(X_data[:, 1]),
         'AP': list(X_data[:, 2]), 'RH': list(X_data[:, 3]), 
         'PE': list(X_data[:, 4])}
X_data = pd.DataFrame(dict1)

a=True
if a:
    general_visual(X_data, RES_DIR+'after_norm/', corr=True)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
X, y = X_data[['AT', 'V', 'AP', 'RH']], X_data[['PE']]
X.shape

X_new = SelectKBest(mutual_info_regression, k=3).fit_transform(X, y)
X_new

In [ ]:
from sklearn.feature_selection import VarianceThreshold

print('{}: {}'.format(0.7, VarianceThreshold(0.7).fit_transform(X_data[['AT', 'V', 'AP', 'RH']])))
print('{}: {}'.format(0.8, VarianceThreshold(0.8).fit_transform(X_data[['AT', 'V', 'AP', 'RH']])))
print('{}: {}'.format(0.9, VarianceThreshold(0.9).fit_transform(X_data[['AT', 'V', 'AP', 'RH']])))
print('{}: {}'.format(0.99, VarianceThreshold(0.99).fit_transform(X_data[['AT', 'V', 'AP', 'RH']])))
print('{}: {}'.format(1.0, VarianceThreshold(1.0).fit_transform(X_data[['AT', 'V', 'AP', 'RH']])))

-----------------------------------------

In [ ]:
from sklearn.model_selection import train_test_split# деление на обуч., вал. и тест.
X, X_val = train_test_split(X_data[['AT', 'V', 'AP', 'RH', 'PE']], test_size=0.4, random_state=21)
X_val, X_test = train_test_split(X_val, test_size=0.25, random_state=11)

print(X_data.shape, X.shape, X_val.shape, X_test.shape)

In [ ]:
get_etr(X)
get_etr(X_val)
get_etr(X_test)

In [ ]:
X_val = scaling(X_val, name='val', a=False)

X = pca_method(X, a=True)
X_val = pca_method(X_val, a = False)
X_test = pca_method(X_test, a=False)

In [ ]:
train_set = pd.concat([X, y], axis=1)
val_set = pd.concat([X_val, y_val], axis=1)
test_set = pd.concat([X_test, y_test], axis=1)

a = True
if a:
    train_set.to_csv(HOME_DIR+'split_data/train_set.csv', index=False)
    val_set.to_csv(HOME_DIR+'split_data/val_set.csv', index=False)
    test_set.to_csv(HOME_DIR+'split_data/test_set.csv', index=False)

In [ ]:
from sklearn.decomposition import FastICA

def ica_method(X_data, a=True):
    Y = list(X_data['PE'])
    ica = FastICA()
    X_data = ica.fit_transform(X=X_data[[feat for feat in X_data.columns[0:4]]])
    dict1 = {'AT': list(X_data[:, 0]), 'V': list(X_data[:, 1]),
             'AP': list(X_data[:, 2]), 'RH': list(X_data[:, 3]),
            'PE': Y}
    X_data = pd.DataFrame(dict1)
    print(X_data.info())

    if a:
        general_visual(X_data, RES_DIR+'after_ica/'+'after_', corr=True)
        general_visual(X_data[['AT', 'V', 'AP', 'RH']], RES_DIR+'after_ica/'+'after_2_', corr=True)
        
    return X_data

In [ ]:
#из def scaling:
dict1 = {}
    if len(feat_list) == 4:
        dict1 = {'AT': list(X_data[:, 0]), 'V': list(X_data[:, 1]),
                 'AP': list(X_data[:, 2]), 'RH': list(X_data[:, 3])}    
        X_data = pd.concat([pd.DataFrame(dict1), X_data['PE']], axis=1)
    else:
        dict1 = {'AT': list(X_data[:, 0]), 'V': list(X_data[:, 1]),
                 'AP': list(X_data[:, 2]), 'RH': list(X_data[:, 3]), 
                 'PE': list(X_data[:, 4])}  
        X_data = pd.DataFrame(dict1)"